#Distributions#
since a Bayesisan network is based on causality and probability the first thing to define are some useful classes to generate a sample from a **distribution**

In [45]:
import random
class Distribution:
    def sample(self) -> int:
        ...

the first thing we define is a class that can give as output just 1 or 0 with the probability 'p' given during initialization

In [46]:
class Bernoulli(Distribution):
    # p = probability of true
    def __init__(self, p:float):
        self.p = p
    def __str__(self):
        return f'p={self.p}'

    def sample(self) -> int:
        return 1 if random.random() < self.p else 0
# for example a coin whose probability of head is 80% will be defined as
unfairCoin = Bernoulli(p=0.8)
head = unfairCoin.sample()
print(f'{"head" if head else "tail"}')
print(unfairCoin)

tail
p=0.8


now we define a multinomial that allows n different outcome, each one with it's own probability.
The result of sample is a nuber from 0 to n-1 representing the index of the outcom as it was given during initializaiton

In [47]:

class Multinomial(Distribution):
    def __init__(self, pList:'list[float]'):
        self.pList = pList
        if sum(pList) < 0.9999 or sum(pList) > 1.000001: #was giving problem with the list [0.7, 0.2, 0.1]
            raise(Exception(f'The sum of the probabilities should be 1 but got {sum(pList)}'))
    def __str__(self):
        return f'pList={self.pList}'

    def sample(self) -> int:
        r = random.random()
        for i, p in enumerate(self.pList):
            if r < p:
                return i
            r -= p
        return len(self.pList)-1

class UniformMultinomial(Multinomial):
    def __init__(self, n:int):
        self.n = n
        self.pList = [1/n]*n

    def __str__(self):
        return f'n={self.n}'

    def sample(self) -> int:
        return random.randint(0, self.n-1)

# representing probability of a dice
dice = UniformMultinomial(n=20)
num = dice.sample() + 1
print(f'dice result:', num if num != 20 else 'crit')

dice result: 12


#Conditional probability table#
now that the basic probability are defined we can put them together to define different probability of an event given different evidence

In [48]:
class CPT:
    '''
    A Fancy Hash table
    '''
    # should allow quick access to the probability of a certain value given the values of the parents
    # take a list of touple: [(id, max1), (id, max2), ...]
    def __init__(self, conditioners:"list['tuple[str, int]']" = []):
        self.conditionersIdOrder = [name for name, _ in conditioners]
        self.table = {}
        self.keys = ['']
        for name, max in conditioners:
            tempKeys = []
            for i in range(max):
                tempKeys += [f'{key}{name}{i}' for key in self.keys]
            self.keys = tempKeys

    def _getKeyFromQuery(self, valuedParents:'dict[str:int]', safeCreation:bool=False)->str:
        '''Get the key from the query'''
        key = ''
        for conditioner in self.conditionersIdOrder:
            try:
                key += f'{conditioner}{valuedParents[conditioner]}'
            except KeyError:
                raise('Not all parents ID are present')
        if safeCreation and (key not in self.keys):
            raise('Key not in the table')
        return key

    def checkComplete(self)->bool:
        '''Check if the table is complete (i.e. all possible values are defined)'''
        return len(self.keys) == len(self.table)

    def setDistribution(self, valuedParents:'dict[str:int]', distribution:Distribution)->None:
        '''Set the distribution for a certain set of parents

        valuedParents: dict (id, value) for the parents that are observed
        distribution: the distribution to use when the parents are observed

        raises 'Not all parents are present' if not all parents are present
        raises 'Key not in the table' if the key is not in the table
        '''
        key = self._getKeyFromQuery(valuedParents)
        self.table[key] = distribution

    def getDistribution(self, valuedParents:'dict[str:int]')->Distribution:
        '''Get the distribution for a certain set of parents

        valuedParents: dictionary (id, value) for the parents that are observed

        raises 'Table is not complete' if the table is not complete
        raises 'Not all parents are present' if not all parents are present
        raises 'Key not in the table' if the key is not in the table
        '''
        if not self.checkComplete():
            raise('Table is not complete')
        key = self._getKeyFromQuery(valuedParents)
        return self.table[key]

##An example of usage##
this is a lot of code, let's se an example of how this thing should be used

In [49]:
# cloud can assume 3 different values
# 0 means no clouds
# 1 means a few clouds
# 2 very cloudy
rain = CPT([('Cloud', 3)])
rain.setDistribution({'Cloud': 0}, Bernoulli(0))
rain.setDistribution({'Cloud': 1}, Bernoulli(0.4))
rain.setDistribution({'Cloud': 2}, Bernoulli(0.9))
print(rain.checkComplete())

True


now that we defined a CPT that represent the probability of raining given an amount of cloud we can use it to get the probability of rain given different observation of 'Cloud'

In [50]:
pRain = rain.getDistribution({'Cloud': 1})
print('Probability of rain given there are some clouds is',pRain)

Probability of rain given there are some clouds is p=0.4


#The basic Node#
we have a good enough abstraction to define the class node that is the basic component of a Bayesian network

In [51]:
class Node:
    def __init__(self, name:str, id:str, parents:list, cpt:CPT):
        self.name = name
        if id is None or id == '':
            raise('id cannot be None or empty')
        self.id = id
        self.parents = parents
        self.cpt = cpt
        self.children = []
        for parent in parents:
            parent.addChild(self)
        #current value of the node
        self.value = None
        #current value of the parents
        self.observed = {}

    def __str__(self):
        return f'{self.name} -> ({self.parents})'

    def addChild(self, child):
        self.children.append(child)

    def _sample(self) -> int:
        self.value = self.cpt.getDistribution(self.observed).sample()
        print(f'{self.name} -> {self.value}')
        for child in self.children:
            child.update(self.id, self.value)

    def update(self, parent:str, value:int):
        self.observed[parent] = value
        if len(self.parents) == len(self.observed):
            # if something does not work maybe
            # make a full check of the parents
            self._sample()

    def reset(self):
        self.value = None
        self.observed = {}

here we have some functions to analize:


1.   `__init__` is the funciton to initialize the node and takes as argument `name` that is useful if we want to keep a description of the node; the `id` is more important because it is used to generate the key of the CPT table, `parents` is a list of nodes that condition the probability of the node, anf the final parameter `cpt`. During the creation if the list of parents is not empty to every parent will be assigned the new node as a children.
2.   `addChild` is the function that assign a chidren to its parent, this function is called automatically so use it explicitly is not recomended since can cause the creation of cicle in the network
3.   `sample` is a fuinction that is automatically called whenever all parents are observed; it generate a state for the node and inform the children about it
4.   `update` is the function that a parent node should call to inform its children of its current sampled state
5.   `reset` just reset a node state



let's se how this is used to create a node and sample a probability (it's not much mode than a wrapper for a CPT)

In [64]:
# fist let's define the probability of the weather to be cloudy
cloudyCPT = CPT()
cloudyCPT.setDistribution(valuedParents={}, distribution=Multinomial([0.7,0.2,0.1]))
cloudyNode = Node('clud amount (weather)', id='Cloud', parents=[], cpt=cloudyCPT)

# now we can create another node, dependent by this one i.e. a child in the graph
rainNode = Node('Rain', id='R', parents=[cloudyNode], cpt=rain)

print('the value of rain is undifined since we haven\'t sampled it yet')
print(rainNode.value)
# now we can sample the nodes 
#(this will sample the whole graph recursively so only the orphan nodes should be sampled)
cloudyNode._sample()

print('After sampling the value of rain is:')
print(rainNode.value)

the value of rain is undifined since we haven't sampled it yet
None
clud amount (weather) -> 1
Rain -> 1
After sampling the value of rain is:
1
